<div style="background-color: #0d0761; border-radius: 20px; color: #fff; font-weight: bold; padding: 10px; text-align:center">
    <h1>Implementar carga completa con validaciones y estrategias avanzadas</h1>
</div>

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Cargar librerías</h2>
</div>

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# crear carpeta para los resultados
import os

ruta_archivos = "../resultados/resultados_dia_4/"
if not os.path.exists(ruta_archivos):
    if not(os.path.exists("../resultados/")): os.mkdir("../resultados/")
    os.mkdir(ruta_archivos)
    print("Carpeta creada")
else:
    print("La carpeta ya existe")

Carpeta creada


<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Crear esquema de base de datos destino</h2>
</div>

In [3]:
# Crear base de datos
conn = sqlite3.connect(ruta_archivos + 'ventas_etl.db')

# Crear tablas con constraints
conn.execute('''
    CREATE TABLE clientes (
        id_cliente INTEGER PRIMARY KEY,
        nombre TEXT NOT NULL,
        email TEXT UNIQUE,
        ciudad TEXT,
        fecha_registro DATE
    )
''')

conn.execute('''
    CREATE TABLE productos (
        id_producto INTEGER PRIMARY KEY,
        nombre TEXT NOT NULL,
        precio REAL NOT NULL,
        categoria TEXT
    )
''')

conn.execute('''
    CREATE TABLE ventas (
        id_venta INTEGER PRIMARY KEY,
        id_cliente INTEGER,
        id_producto INTEGER,
        cantidad INTEGER NOT NULL,
        precio_unitario REAL NOT NULL,
        fecha_venta DATE,
        FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente),
        FOREIGN KEY (id_producto) REFERENCES productos(id_producto)
    )
''')

conn.commit()

<div style="border:2px solid #0d0761; width:620px; margin:auto; text-align:center">
<h4 style="font-weight:bold; color:#0d0761">DIAGRAMA ENTIDAD RELACIÓN</h4>
<img src="../imagenes/diagrama_ER.png" alt="Diagrama Entidad Relación" width="600px" />
</div>

Es importante recordar que SQLite no cuenta con un tipo de dato "date" nativo; en su lugar lo convierte a "text" por afinidad.

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Crear datos de ejemplo para carga</h2>
</div>

In [4]:
# Datos de clientes
clientes_df = pd.DataFrame({
    'id_cliente': range(1, 6),
    'nombre': ['Ana García', 'Carlos López', 'María Rodríguez', 'Juan Pérez', 'Luis Martín'],
    'email': ['ana@email.com', 'carlos@email.com', 'maria@email.com', 'juan@email.com', 'luis@email.com'],
    'ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Sevilla'],
    'fecha_registro': pd.date_range('2023-01-01', periods=5, freq='MS')
})

# Datos de productos
productos_df = pd.DataFrame({
    'id_producto': range(101, 106),
    'nombre': ['Laptop', 'Mouse', 'Teclado', 'Monitor', 'Audífonos'],
    'precio': [1200, 25, 80, 300, 150],
    'categoria': ['Electrónica', 'Accesorios', 'Accesorios', 'Electrónica', 'Audio']
})

# Datos de ventas (con algunos errores intencionales)
np.random.seed(42)
ventas_df = pd.DataFrame({
    'id_venta': range(1, 21),
    'id_cliente': np.random.choice(range(1, 8), 20),  # Algunos IDs inexistentes
    'id_producto': np.random.choice(range(101, 108), 20),  # Algunos IDs inexistentes
    'cantidad': np.random.randint(1, 5, 20),
    'precio_unitario': np.random.choice([1200, 25, 80, 300, 150], 20),
    'fecha_venta': pd.date_range('2024-01-01', periods=20, freq='D')
})

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Implementar carga con validaciones</h2>
</div>

In [5]:
# Función para cargar con validaciones
def cargar_con_validacion(df, tabla, conn, claves_foraneas=None):
    try:
        # Validar claves foráneas si se especifican
        if claves_foraneas:
            for columna, tabla_ref, columna_ref in claves_foraneas:
                valores_validos = pd.read_sql(f'SELECT {columna_ref} FROM {tabla_ref}', conn)
                valores_validos = valores_validos[columna_ref].tolist()
                
                invalidos = ~df[columna].isin(valores_validos)
                if invalidos.any():
                    print(f"Advertencia: {invalidos.sum()} registros en {columna} no existen en {tabla_ref}")
                    # Opción: filtrar inválidos o marcar como NULL
                    df = df[~invalidos]  # Filtrar inválidos
        
        # Cargar datos
        df.to_sql(tabla, conn, index=False, if_exists='append')
        print(f"✓ Cargados {len(df)} registros en {tabla}")
        return True
        
    except Exception as e:
        print(f"✗ Error cargando {tabla}: {e}")
        return False

# Cargar tablas base (sin dependencias)
exito_clientes = cargar_con_validacion(clientes_df, 'clientes', conn)
exito_productos = cargar_con_validacion(productos_df, 'productos', conn)

# Cargar ventas con validaciones de FK
if exito_clientes and exito_productos:
    claves_ventas = [
        ('id_cliente', 'clientes', 'id_cliente'),
        ('id_producto', 'productos', 'id_producto')
    ]
    cargar_con_validacion(ventas_df, 'ventas', conn, claves_ventas)

✓ Cargados 5 registros en clientes
✓ Cargados 5 registros en productos
Advertencia: 5 registros en id_cliente no existen en clientes
Advertencia: 4 registros en id_producto no existen en productos
✓ Cargados 11 registros en ventas


<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Verificar carga y ejecutar consultas</h2>
</div>

In [6]:
# Verificar conteos
for tabla in ['clientes', 'productos', 'ventas']:
    count = pd.read_sql(f'SELECT COUNT(*) FROM {tabla}', conn).iloc[0,0]
    print(f"{tabla}: {count} registros")

# Consulta de ejemplo: ventas por cliente
query_result = pd.read_sql('''
    SELECT c.nombre, COUNT(v.id_venta) as num_ventas, 
           SUM(v.cantidad * v.precio_unitario) as total_ventas
    FROM clientes c
    LEFT JOIN ventas v ON c.id_cliente = v.id_cliente
    GROUP BY c.id_cliente, c.nombre
    ORDER BY total_ventas DESC
''', conn)

print("\nVentas por cliente:")
print(query_result)

conn.close()

clientes: 5 registros
productos: 5 registros
ventas: 11 registros

Ventas por cliente:
            nombre  num_ventas  total_ventas
0      Luis Martín           5        4400.0
1  María Rodríguez           4         765.0
2       Juan Pérez           1         300.0
3     Carlos López           1         100.0
4       Ana García           0           NaN


<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Conclusión</h2>
</div>

Las validaciones de claves foráneas funcionaron y la integridad referencial está intacta, pero los constraints PRIMARY KEY, UNIQUE y NOT NULL no se validaron y manejaron antes de la inserción, porque los datos de entrada ya eran válidos.

Por otra parte, hay problemas de consistencia en los precios unitarios entre las tablas de productos y ventas (que no tiene relación con la integridad referencial), como se muestra a continuación:

In [7]:
# con context manager

query = '''
    SELECT 
        p.id_producto, 
        p.nombre, 
        p.precio AS precio_en_productos, 
        GROUP_CONCAT(DISTINCT v.precio_unitario ORDER BY v.precio_unitario ASC) AS listado_precios_en_ventas
    FROM productos p
    INNER JOIN ventas v ON v.id_producto = p.id_producto
    GROUP BY p.id_producto
'''

with sqlite3.connect(ruta_archivos + 'ventas_etl.db') as conn:
    df_sql = pd.read_sql(query, conn)
    print(df_sql)

   id_producto     nombre  precio_en_productos listado_precios_en_ventas
0          101     Laptop               1200.0                80.0,150.0
1          102      Mouse                 25.0                      25.0
2          103    Teclado                 80.0              300.0,1200.0
3          104    Monitor                300.0                      25.0
4          105  Audífonos                150.0                25.0,300.0


Vemos que el producto Laptop tiene dos precios diferentes en la tabla de ventas, y otro precio distinto en la tabla de productos. Eso no es consistente, salvo que los precios en la tabla de ventas reflejen, por ejemplo, precios históricos o promocionales.